
# Serie de concurrencia de Clojure.
### 1. Vars: indirección, código recargable, enlace raíz, variables dinámicas
### 2. atoms: simples, más usados, reintentados, atómicos, descoordinados
### 3. refs: punto de venta original, coordinado y sincrónico, pero que rara vez se utiliza hoy en día.
### 4. agents: asíncronos, descoordinados, seguros para los efectos secundarios (por ejemplo, registro en el archivo),
### se pueden combinar con STM, ejecutarse solo cuando la transacción de referencia se realiza correctamente


# Atoms: simple, más utilizado, reintentado, atómico, descoordinado

In [1]:
(def goleadores (atom [{:name "Messi" :scores 0}  {:name "Ronaldo" :scores 0}]))
@goleadores

[{:name "Messi", :scores 0} {:name "Ronaldo", :scores 0}]

In [2]:
(deref goleadores)

[{:name "Messi", :scores 0} {:name "Ronaldo", :scores 0}]

In [4]:
(dotimes [n 10]
(swap! goleadores update-in [0 :scores] + (rand-int 30))
(swap! goleadores update-in [1 :scores] + (rand-int 30))
)
@goleadores

[{:name "Messi", :scores 283} {:name "Ronaldo", :scores 217}]


# refs: coordinado, sincrónico, punto de inicio original pero rara vez utilizado

In [22]:
(def terminators (ref {:t1 0 :t2 0 :t800 1}))
@terminators


{:t1 0, :t2 0, :t800 1}

In [23]:
(def aliens (ref {:soldier 0 :queen 1}))
@aliens

{:soldier 0, :queen 1}

In [97]:
@terminators

{:t1 30, :t2 30, :t800 31}


# Agentes: asíncronos, descoordinados, seguro para efectos secundarios (por ejemplo, registro en archivo), se puede combinar con STM (ejecutado al final)

In [102]:
(def you_got_mail (agent []))

#'user/you_got_mail

In [103]:
(def contador (ref 0))

#'user/contador

In [109]:
(dosync
         (println "Inicia Update")
 
         (send you_got_mail #(do (println "Enviar correo...")
                                 (conj % (str "mail" @contador) )))
 
         (terminators :t4 "T10000")
         (aliens :alien "alien")
         (alter contador inc)
         (Thread/sleep 3000)
         (println "Updated"))

Inicia Update
Enviar correo...
Updated


nil

In [110]:
@you_got_mail

["mail1" "mail2" "mail2" "mail2"]

## Funciones de una agente:
1. agent
Un agente se crea mediante el comando de agente.

2. send
Esta función se utiliza para enviar un valor al agente.

3. shutdown-agents
Esta función se utiliza para apagar cualquier agente en ejecución.

4. send-off
Hay casos en los que a un agente se le asigna una función que es de naturaleza bloqueante.

5. await-off
Dado que hay una demora cuando se actualiza un valor de un agente, Clojure proporcionó una función de "espera" que se usa para especificar el tiempo en milisegundos para esperar a que se actualice el agente.

6. await
Bloquea el hilo actual (¡indefinidamente!) Hasta que se hayan realizado todas las acciones despachadas hasta el momento, desde este hilo o agente, al agente (s). Bloqueará a los agentes fallidos.

7. agent-error
Devuelve la excepción lanzada durante una acción asincrónica del agente, si el agente falla. Devuelve nil si el agente no falla.


# Vars indirección, código recargable, variables dinámicas.
### puedes crear una var dinámica cuyo enlace se puede cambiar.


In [11]:
(def batman "Primer Batman")

#'user/batman

In [113]:
batman

"Primer Batman"

In [12]:
(def ^:dynamic batman "Adam West")

#'user/batman

In [13]:
(binding [batman "Micheal Keaton"]
  (println batman)
    (Thread/sleep 1000)
    
 (binding [batman "Val Kilmer"]
  (println batman)
    (Thread/sleep 1000))
 (binding [batman "George Clooney"]
  (println batman)
    (Thread/sleep 1000))    

(binding [batman "Christian Bale"]
  (println batman)
    (Thread/sleep 1000))
  )  

Micheal Keaton
Val Kilmer
George Clooney
Christian Bale


nil

# Ejecicios:

## Ejecicio 01: Crea un Chat, que intercambie mensajes aleatorios entre dos personas. Cada persona debe tener 10 posibles mensajes de tipo string. Que debes de usar?

## Ejecicio 02: Crea un bitacora de errores de web desde Hawkins: 404, 500, 502, 504 y más. Tu operacion debe simular un stack de n posibles errores.